In [5]:
from bs4 import BeautifulSoup
import requests as requests
import time
import re
from datetime import datetime
import pandas as pd
import numpy as np

In [3]:
## Key variables valid for all the chuncks
timestamp_pattern = re.compile(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z)')
original_format = "%Y-%m-%dT%H:%M:%S.%fZ"
desired_format = "%Y-%m-%d %H:%M:%S"


## Test chunck
NB: need to solve the problem of some items not having a title (these are the comments). At the current time, I solved the problem with a `try - except`.

In [4]:
titles = []

start = time.time()

for i in range(1,11):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue

end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")

## Collect post header with all the related data

In [47]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
banner = soup.find('div', class_='item_other__MjgP3')

#post_stack = banner.find('span').text

banner_data = []

for i in banner.find_all('span'):
    banner_data.append(i.text)
    
#n_comments = soup.find('a', class_='text-reset position-relative').get_text()
#nym_creator = banner.find('a', ) 

print([titolo, banner_data])


['El Salvador Makes Bitcoin Legal Tender', ['603 sats', ' \\ ', '4 boost', ' \\ ', ' \\ ', '@k00b  11 Jun 2021', ' ', '', ' ', 'bitcoin', '']]


## Collect timestamp of post creation
The following code scans the parsed html response looking for an `<a>` tag that has an attribute `title` matching the timestamp format in the website.
Then it extracts it and turns it into a datetime object. 

In [20]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

timestamp_element = soup.find('a', attrs={'title': timestamp_pattern})
timestamp_match = timestamp_pattern.search(timestamp_element['title']).group(1)

timestamp_datetime = datetime.strptime(timestamp_match, original_format)

print(timestamp_datetime)


2021-06-11 19:26:02.662000


# Scraping Items of stacker.news

In [7]:
df = pd.DataFrame(columns=['titolo', 'numero commenti', 'boost', 'sats', 'betha'])
n_posts = range(1,20)

for number in n_posts:
    print("Doing", number)
    url_posts = f'https://stacker.news/items/{number}'
    response = requests.get(url_posts)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        contenuto=soup.find('div', class_="item_fullItemContainer__ZAYtZ").get_text()
    except:
        continue
    try:
        titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
    except:
        continue
    try:
        banner = soup.find('div', class_='item_other__MjgP3')
    except:
        continue
    try:
        commento = soup.find('a', class_='text-reset position-relative').get_text()
    except:        
        continue

    banner_data = []
    for i in banner.find_all('span'):
        banner_data.append(i.text)
    index=number-1
    for b in banner_data:
        if "boost" in b:
            df.at[index, 'boost'] = b
        if "sats" in b:
            df.at[index, 'sats'] = b
        if "@" in b:
            df.at[index, 'betha'] = b

    df.at[index, 'titolo'] = titolo
    df.at[index, 'numero commenti'] = commento
    df.at[index, 'contenuto'] = contenuto

df['author'] = df['betha'].str.extract(r'@(\w+)')
df['data'] = df['betha'].str.extract(r'(\d+\s\w+\s\d+)')
df.drop('betha', axis=1, inplace=True)

df


Doing 1


/tmp/ipykernel_2556/2015901663.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'contenuto'] = contenuto


Doing 2
Doing 3
Doing 4
Doing 5
Doing 6
Doing 7
Doing 8
Doing 9
Doing 10
Doing 11
Doing 12
Doing 13
Doing 14
Doing 15
Doing 16
Doing 17
Doing 18
Doing 19


,titolo,numero commenti,boost,sats,contenuto,author,data
0,El Salvador Makes Bitcoin Legal Tender,13 comments,4 boost,603 sats,,k00b,11 Jun 2021
1,stacker news soft launch AMA,30 comments,NaN,2 sats,Figured I'd make a place to raise questions ab...,k00b,11 Jun 2021
7,The Lightning Conference 2019: Alex Bosworth,2 comments,NaN,0 sats,,k00b,11 Jun 2021


# Scraping user profiles
Users profiles are scraped starting from the list of users extracted by scraping all the items (posts+comments)
The link to get the user profile is `https://stacker.news/$username$` 

## Modularize the code - user profile scraping
Define functions to modularize and simplify the user profile scraping.
The functions are defined in [](scraping_user.py) and have been tested with [](test_scraping_user.py). To run the tests open a the terminal/CMD and run the script [](/tests/test_scraping_user.py) from there, running it from a JupyterNotebook could raise errors.
The testing script tested every function in some corner cases (missing data/request error). 